## Data Parallel Training for ThirdAI's UDT

This notebook shows how to run Data Parallel Training for ThirdAI's UDT. We will be using CLINC 150 small dataset for training and evaluation for this demo. But, you can easily replace this with your workload. 

ThirdAI's Distributed Data Parallel Training assumes that you already have a ray cluster running. For this demo, we would be using the ray mock cluster to simulate the ray cluster. For seetting up a ray cluster, see here: https://docs.ray.io/en/latest/cluster/getting-started.html

In [ ]:
!pip3 install thirdai --upgrade
!pip3 install ray

import thirdai
thirdai.licensing.activate("HN7J-W79C-KN9U-WTKE-9PNM-4PVR-CNPJ-WTWE")

## Ray Cluster Initialization
For the purpose of this demo, we will be initializing a mock ray cluster here.

In [ ]:
from ray.cluster_utils import Cluster

mini_cluster = Cluster(
    initialize_head=True,
    head_node_args={
        "num_cpus": 1,
    },
)
mini_cluster.add_node(num_cpus=1)

# Dataset Download

We will use the demos module in the thirdai package to download the CLINC 150 small dataset. You can replace theis step and the next step with a download method and a UDT initialization that is specific to yourd ataset.

In [ ]:
from thirdai.demos import download_clinc_dataset

train_filenames, test_filename , _ = download_clinc_dataset(num_training_files=2, clinc_small=True)

# UDT Initialization
We can now create a UDT model by passing in the types of each column in the dataset and the target column we want to be able to predict.

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "text": bolt.types.text(),
        "category": bolt.types.categorical(),
    },
    target="category",
    n_target_classes=151,
    integer_target=True,
)

## Distributed Training

We will now train a UDT model in distributed data parallel fashion. Feel free to customize the number of epochs and the learning rate; we have chosen values that give good convergence. 

In [ ]:


import thirdai.distributed_bolt as dist_bolt
import os

cluster_config = dist_bolt.RayTrainingClusterConfig(
    num_workers=2,
    cluster_address=mini_cluster.address,
    requested_cpus_per_node=1,
    communication_type="linear",
    ignore_reinit_error=True,
)



model.train_distributed(
        cluster_config=cluster_config,
        filenames=train_filenames,
        batch_size=256,
        epochs=1,
        learning_rate=0.02,
        metrics=["mean_squared_error"],
        verbose=True,
    )

# Evaluation
Evaluating the performance of the UDT model is just one line!

In [ ]:
model.evaluate(test_filename, metrics=["categorical_accuracy"]);